# Framework Comparison

- Numpy vs Tensorflow vs PyTorch
- 같은 연산이 각각 어떻게 구동이 되는지 알아보고 속도 역시 비교해보도록 하겠습니다.
- x * y + z

In [1]:
# 연산에 필요한 numpy, 시간을 측정하기 위해 datetime을 불러옵니다.
import numpy as np 
from datetime import datetime
start = datetime.now()

# 랜덤하게 3x4 형태의 변수 x,y,z를 설정해줍니다.
np.random.seed(0)

N,D = 3,4

x = np.random.randn(N,D)
y = np.random.randn(N,D)
z = np.random.randn(N,D)

# x,y,z를 이용해 x*y+z를 계산해줍니다.
a = x * y
b = a + z
c = np.sum(b)

# 기울기(gradient)가 1이라고 가정하고 역전파를 해줍니다. 역전파에 대한 내용은 4장에서 자세히 다룹니다.
grad_c = 1.0
grad_b = grad_c * np.ones((N,D))
grad_a = grad_b.copy()
grad_z = grad_b.copy()
grad_y = grad_a * y
grad_x = grad_a * x

# 각각의 기울기가 몇인지 걸린 시간은 얼마인지 확인해봅니다.
print(grad_x)
print(grad_y)
print(grad_z)
print(datetime.now()-start)

[[ 1.76405235  0.40015721  0.97873798  2.2408932 ]
 [ 1.86755799 -0.97727788  0.95008842 -0.15135721]
 [-0.10321885  0.4105985   0.14404357  1.45427351]]
[[ 0.76103773  0.12167502  0.44386323  0.33367433]
 [ 1.49407907 -0.20515826  0.3130677  -0.85409574]
 [-2.55298982  0.6536186   0.8644362  -0.74216502]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
0:00:00.002931


In [3]:
# 이번에는 텐서플로 프레임워크를 이용해 같은 연산을 해보도록 하겠습니다.
import tensorflow as tf
import numpy as np
from datetime import datetime
start = datetime.now()

# 텐서플로는 연산 그래프를 먼저 정의하고 추후에 여기에 값을 전달하는 방식입니다. 여기서는 비어있는 그래프만 정의해줍니다.
# Define Graph on GPU
with tf.device('/gpu:0'):              # 해당 연산을 0번 gpu에서 하겠다는 의미입니다.
    x = tf.placeholder(tf.float32)     # 비어있는 노드인 placeholder를 정의하고 여기에 들어가는 데이터타입을 명시 해놓습니다.
    y = tf.placeholder(tf.float32)
    z = tf.placeholder(tf.float32)

    a = x * y                          # 연산 과정 또한 정의해줍니다.
    b = a + z
    c = tf.reduce_sum(b)
    
grad_x, grad_y, grad_z = tf.gradients(c,[x,y,z])  # c에 대한 x,y,z의 기울기(gradient)를 구하고 이를 각각 grad_x, grad_y, grad_z에 저장하도록 지정해놓습니다.

# 실제적인 계산이 이루어지는 부분. 텐서플로에서는 이를 세션이라고 합니다.
with tf.Session() as sess:
    values = {
        x: np.random.randn(N,D),     # 여기서 실제 값들이 생성됩니다.
        y: np.random.randn(N,D),
        z: np.random.randn(N,D)           
    }
    out = sess.run([c,grad_x,grad_y,grad_z],feed_dict = values)  # 세션에서 실제로 값을 계산하는 부분입니다. feed_dict를 통해서 값들을 전달합니다.
    c_val, grad_x_val, grad_y_val, grad_z_val = out

# 값들을 확인하고 걸린 시간을 측정합니다.
print(grad_x_val)
print(grad_y_val)
print(grad_z_val)
print(datetime.now()-start)

[[ 1.4882522   1.8958892   1.1787796  -0.17992483]
 [-1.0707526   1.0544517  -0.40317693  1.222445  ]
 [ 0.20827498  0.97663903  0.3563664   0.7065732 ]]
[[ 1.1394007  -1.2348258   0.40234163 -0.6848101 ]
 [-0.87079716 -0.5788497  -0.31155252  0.05616534]
 [-1.1651498   0.9008265   0.46566245 -1.5362437 ]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
0:00:00.055856


In [5]:
# 이번에는 파이토치를 이용해 같은 연산을 진행해보도록 하겠습니다.
import torch
from datetime import datetime
start = datetime.now()

N,D = 3,4

# x,y,z를 랜덤하게 초기화 해줍니다. 
# https://pytorch.org/docs/stable/torch.html?highlight=randn#torch.randn
x = torch.randn(N,D,device=torch.device('cuda'), requires_grad=True)
y = torch.randn(N,D,device=torch.device('cuda'), requires_grad=True)
z = torch.randn(N,D,device=torch.device('cuda'), requires_grad=True)

# 연산 그래프는 정의됨과 동시에 연산됩니다.
a = x * y
b = a + z
c = torch.sum(b)

# 기울기(gradient)가 1.0라고 가정하고 최종 값인 c에서 backward를 통해 역전파를 해줍니다.
# 넘파이와 비교했을때 이 과정이 자동적으로 게산되는 것을 확인할 수 있습니다.
c.backward(gradient=torch.cuda.FloatTensor([1.0]))

# 각각의 기울기와 걸린 시간을 출력합니다.
print(x.grad)
print(y.grad)
print(z.grad)
print(datetime.now()-start)

tensor([[ 0.7840, -2.7837,  0.9255,  0.8922],
        [ 0.3718, -0.5948,  0.6418, -0.6967],
        [-0.2106, -0.6479, -0.7273,  0.5540]], device='cuda:0')
tensor([[-1.1159,  0.4730, -2.2700, -1.4020],
        [ 0.2738, -1.5292, -1.6174,  0.8994],
        [ 0.0773,  1.6193,  0.9806, -0.8056]], device='cuda:0')
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0')
0:00:00.006936
